In [11]:
from google.colab import auth
auth.authenticate_user()
       

In [12]:
from google.auth import default
import gspread
creds, _ = default()
gc = gspread.authorize(creds)        

In [13]:
worksheet = gc.open('hello').sheet1
list_of_dicts = worksheet.get_all_records()
print(list_of_dicts)

            

[{'Timestamp': '5/8/2023 13:01:04', "User ID (Don't edit this)": 'b5711006-f9ef-4171-a6c7-1e1852f89608', 'Name': 'iliyas', 'Amazon Sales Dashboard Screenshot Drive link \n(You can paste the drive link here)': 'https://drive.google.com/file/d/1NcWx46LzpTTtoNLLqG01_jtl_5IKlicl/view?usp=share_link', 'You can also share your key insights and recommendations': 'Amazon Sales Dashboard Which Contains the Total Sales and Profit by different category and month, year ', 'Percentage ': '', 'Grade': '', 'Solution Url ': 'https://drive.google.com/file/d/13vJuRFuMY8tQByl56gZIL1rL-9TtzxaL/view'}, {'Timestamp': '5/8/2023 13:01:34', "User ID (Don't edit this)": 'a4f31c11-b4a1-4ec9-ae49-aae0509097db', 'Name': 'Ajay Samudrala', 'Amazon Sales Dashboard Screenshot Drive link \n(You can paste the drive link here)': 'https://drive.google.com/file/d/12OrIlNHuGbAOYZAkTkbVhWTTJ_mnUh2H/view?usp=sharing', 'You can also share your key insights and recommendations': '', 'Percentage ': '', 'Grade': '', 'Solution Url

In [14]:
import requests
from PIL import Image
import numpy as np
from skimage.metrics import structural_similarity as ssim


In [15]:
#grade assigning 
def assign_grade(similarity_percentage):
    if similarity_percentage==0:
      grade='N'
    elif similarity_percentage >= 90:
        grade = 'A'
    elif similarity_percentage >= 80:
        grade = 'B'
    elif similarity_percentage >= 70:
        grade = 'C'
    elif similarity_percentage >= 60:
        grade = 'D'
    else:
        grade = 'F'
    return grade

In [16]:
##Image Comparison Logic
def calculate_image_similarity(submission_image_path, solution_image_path):
    solution_image = Image.open(solution_image_path)
    # Open the images using PIL
    try:
      submission_image = Image.open(submission_image_path)
    except IOError:
      return 0
    # Resize both images to the same dimensions
    max_dimension = max(submission_image.width, submission_image.height, solution_image.width, solution_image.height)
    submission_image_resized = submission_image.resize((max_dimension, max_dimension))
    solution_image_resized = solution_image.resize((max_dimension, max_dimension))

    # Convert images to grayscale for comparison
    submission_image_gray = submission_image_resized.convert('L')
    solution_image_gray = solution_image_resized.convert('L')

    # Calculate the similarity between the images using the SSIM metric
    similarity_score = ssim(np.array(submission_image_gray), np.array(solution_image_gray), data_range=submission_image_gray.getextrema()[1])

    return similarity_score

In [17]:
#convert the drive link

def convert_drive_link(input_link):
    file_id = input_link.split('/')[-2]
    download_link = f'https://drive.google.com/uc?export=download&id={file_id}'
    return download_link


In [18]:
#downloading the file

def download_file(url, filename):
    response = requests.get(url)
    with open(filename, 'wb') as file:
        file.write(response.content)

In [19]:
#main function 
import pandas as pd
df = pd.DataFrame(list_of_dicts)
my_len=len(df.iloc[:,3])
my_list_percentage=[]
my_list_grade=[]

for value in df.iloc[1:,3]:
  #converting the drive link

  submission_drive_link_url = convert_drive_link(value)
  solution_drive_link_url = convert_drive_link(df.iloc[0,7])
  print(solution_drive_link_url)

  #downloading the image
  download_file(submission_drive_link_url, 'submission_screenshot.png')
  download_file(solution_drive_link_url, 'solution_screenshot.png')

  #calculating the similarity
  similarity_score = calculate_image_similarity('submission_screenshot.png', 'solution_screenshot.png')
  similarity_percentage = similarity_score * 100

  #assigning the grade
  grade = assign_grade(similarity_percentage)

  #appending the values to lists
  my_list_percentage.append(similarity_percentage)
  my_list_grade.append(grade)

https://drive.google.com/uc?export=download&id=13vJuRFuMY8tQByl56gZIL1rL-9TtzxaL
https://drive.google.com/uc?export=download&id=13vJuRFuMY8tQByl56gZIL1rL-9TtzxaL
https://drive.google.com/uc?export=download&id=13vJuRFuMY8tQByl56gZIL1rL-9TtzxaL
https://drive.google.com/uc?export=download&id=13vJuRFuMY8tQByl56gZIL1rL-9TtzxaL
https://drive.google.com/uc?export=download&id=13vJuRFuMY8tQByl56gZIL1rL-9TtzxaL
https://drive.google.com/uc?export=download&id=13vJuRFuMY8tQByl56gZIL1rL-9TtzxaL


In [21]:
#converting the values to a dataframe
df1=pd.DataFrame()
df1['Percentage'] = my_list_percentage
df1['Grade']=my_list_grade

print(df1['Percentage'],df1['Grade'])

0    48.328991
1     0.000000
2    47.038866
3    55.258979
4    53.070580
5    53.070580
Name: Percentage, dtype: float64 0    F
1    N
2    F
3    F
4    F
5    F
Name: Grade, dtype: object
